Mark Oulitin 208283291
Itay Cohen 211896261

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot
from sklearn.model_selection import train_test_split

In [4]:
loans = pd.read_csv('input/should-this-loan-be-approved-or-denied/SBAnational.csv')
loans.head()

FileNotFoundError: [Errno 2] No such file or directory: 'input/should-this-loan-be-approved-or-denied/SBAnational.csv'

In [ ]:
loans.shape

Data Preparation

In [ ]:
# we want to make a binary value stating if the loan was defaulted or not
loans['Defaulted'] = [1 if app == 'CHGOFF' else 0 for app in loans.MIS_Status.values]
loans = loans.drop(columns=['MIS_Status'])

# we only take the loans created before 2007
loans['ApprovalFY'] = loans['ApprovalFY'].replace({'A':'','B':''}, regex = True).astype(int)
loans.drop(loans[loans['ApprovalFY']>2007].index, axis = 0, inplace = True)

loans = loans.drop(columns=['ApprovalFY','ApprovalDate','DisbursementDate','ChgOffDate','LoanNr_ChkDgt','Name','Zip', 'City'])

In [ ]:
loans.head()

In [ ]:
loans.shape

In [ ]:
def amount_to_float(number):
    num = number.replace("$", "")
    num = num.replace(",","")
    num = num.replace(" ","")
    return float(num)

#instead of dealing with money as a string we will deal with it as a number
loans['BalanceGross'] = loans['BalanceGross'].apply(lambda x: amount_to_float(x))
loans['DisbursementGross'] = loans['DisbursementGross'].apply(lambda x: amount_to_float(x))
loans['ChgOffPrinGr'] = loans['ChgOffPrinGr'].apply(lambda x: amount_to_float(x))
loans['GrAppv'] = loans['GrAppv'].apply(lambda x: amount_to_float(x))
loans['SBA_Appv'] = loans['SBA_Appv'].apply(lambda x: amount_to_float(x))

In [ ]:
# we replace it to be a binary where 1 to be a new business and 0 an old business
loans['NewExist'] = loans['NewExist'].replace(1,0)
loans['NewExist'] = loans['NewExist'].replace(2,1)

In [ ]:
# we replace them both to be binary instead of Y and N
loans['LowDoc'] = loans['LowDoc'].replace({'Y':'1', 'N':'0'}, regex=True)
valid = ['1', '0']
loans = loans.loc[loans['LowDoc'].isin(valid)]
loans['LowDoc'] = loans['LowDoc'].astype(int)
loans['RevLineCr'] = loans['RevLineCr'].replace({'Y':'1', 'N':'0'}, regex=True)
loans = loans.loc[loans['RevLineCr'].isin(valid)]
loans['RevLineCr'] = loans['RevLineCr'].astype(int)

In [ ]:
# we make franchise a binary value where 1 is true and 0 is false
loans['FranchiseCode'] = loans['FranchiseCode'].replace(1,0)
loans['FranchiseCode'] = np.where((loans.FranchiseCode != 0),1,loans.FranchiseCode)
loans.rename(columns={"FranchiseCode":"Franchise"},inplace=True)

In [ ]:
loans.shape

In [ ]:
loans = pd.get_dummies(loans,columns=['BankState','State'],prefix=['BankState','State'])

In [ ]:
#only take the first 2 digits from the NAICS values
def first_n_digits(number, n):
    return int(str(number)[:n])
loans['NAICS'] = loans['NAICS'].apply(lambda x: first_n_digits(x,2))

In [ ]:
loans.head()

In [ ]:
loans['NAICS'].unique()

In [ ]:
loans.dtypes

Modeling

In [ ]:
y = loans['Defaulted']
X = loans.drop(columns=['Defaulted'])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

In [ ]:
# number of input columns
n_inputs = X.shape[1]
# define encoder
visible = Input(shape=(n_inputs,))
# encoder level 1
e = Dense(100)(visible)
e = BatchNormalization()(e)
e = LeakyReLU()(e)
# encoder level 2
e = Dense(50)(e)
e = BatchNormalization()(e)
e = LeakyReLU()(e)
# encoder level 3
e = Dense(15)(e)
e = BatchNormalization()(e)
e = LeakyReLU()(e)
# bottleneck
n_bottleneck = round(float(n_inputs) / 2.0)
bottleneck = Dense(n_bottleneck)(e)

In [ ]:
# define decoder, level 1
d = Dense(50)(bottleneck)
d = BatchNormalization()(d)
d = LeakyReLU()(d)
# decoder level 2
d = Dense(100)(d)
d = BatchNormalization()(d)
d = LeakyReLU()(d)
# decoder level 3
d = Dense(n_inputs)(d)
d = BatchNormalization()(d)
d = LeakyReLU()(d)
# output layer
output = Dense(n_inputs, activation='linear')(d)

In [ ]:
# define autoencoder model
model = Model(inputs=visible, outputs=output)
# compile autoencoder model
model.compile(optimizer='adam', loss='mse')
# plot the autoencoder
plot_model(model, 'autoencoder_compress.png', show_shapes=True)
# fit the autoencoder model to reconstruct input
history = model.fit(X_train, X_train, epochs=200, batch_size=16, verbose=2, validation_data=(X_test,X_test))